In [20]:
# !pip install numpy

In [21]:
# import serial
# import time
# import struct

# COM = 'COM7'

# ser = serial.Serial(
#     port=COM,
#     baudrate=9600,
#     parity=serial.PARITY_NONE,
#     stopbits=serial.STOPBITS_ONE,
#     bytesize=serial.EIGHTBITS,
#     timeout = 1
# )

In [22]:
# st="\x23\x30\x34\x0D"
# x=st.encode()

In [23]:
# request = "$02Y(cr)\r"
# request = bytes(request, encoding="latin1")

In [24]:
# ser.write(request)
# print("write: ",request)
# time.sleep(1)
# out = ser.read(17)
# # out = str(out,encoding = "latin1")
# time.sleep(2)
# ser.close()
# out
# # text=ser.readline()
# # temp=text.decode('utf-8')
# # print("response: ",temp)

In [25]:
# import pymodbus
# from pymodbus.client.sync import ModbusSerialClient as ModbusClient
# from pymodbus.transaction import ModbusRtuFramer
# import time

# from pymodbus.constants import Defaults
# Defaults.RetryOnEmpty=True
# Defaults.Timeout= 0.25
# Defaults.Retries= 3

# client = ModbusClient(method='rtu', port='COM10',stopbits = 2, bytesize = 8, parity = 'N', baudrate= 9600)
# # client=ModbusClient('10.22.29.60', port=5000, framer=ModbusRtuFramer)
# connection = client.connect()
# for x in range (1,8,1):
#     regPlc = client.read_holding_registers(address=x, count=1, unit=0x01) #baca data word/analog dari address 1000 ke 1019
#     time.sleep(1)
#     print(regPlc)
#     intsvt = int(regPlc.registers[0])
#     print(intsvt)
# client.close()

In [26]:
# import minimalmodbus
# import serial
# import numpy as np
# import struct

# def u16_to_float(data):
#     temp = data & 0x0FFF
#     temp /= 16.0
#     if data & 0x1000:
#         temp -= 256
#     return temp

# def unpack(data):
#     val = struct.pack('H', data)
#     temp = struct.unpack('h', val)[0]
#     return val

# instrument = minimalmodbus.Instrument ('COM6', 1)
# instrument.serial.baudrate = 9600
# instrument.serial.bytesize = 8
# instrument.serial.parity = serial.PARITY_NONE
# instrument.serial.stopbits = 1
# instrument.serial.timeout = 1
# instrument.mode = minimalmodbus.MODE_RTU
# instrument.close_port_after_each_call = True
# instrument.debug = True

# data = instrument.read_register (0x0008,0, 0x02, signed = False)
# print("chanel:" + str(i), "data :" + str(data) + "  ==  " + str(u16_to_float(data)*-1) +"  ==  " +str(unpack(data)))

In [27]:
# def float_to_u16(temp):
#     return int(temp * 16) & 0x1FFF

# float_to_u16(22)

##wp8028adam

In [28]:
import minimalmodbus
import serial
import time
import struct

COM = 'COM7'

In [29]:
_CRC16TABLE = (
    0,
    49345,
    49537,
    320,
    49921,
    960,
    640,
    49729,
    50689,
    1728,
    1920,
    51009,
    1280,
    50625,
    50305,
    1088,
    52225,
    3264,
    3456,
    52545,
    3840,
    53185,
    52865,
    3648,
    2560,
    51905,
    52097,
    2880,
    51457,
    2496,
    2176,
    51265,
    55297,
    6336,
    6528,
    55617,
    6912,
    56257,
    55937,
    6720,
    7680,
    57025,
    57217,
    8000,
    56577,
    7616,
    7296,
    56385,
    5120,
    54465,
    54657,
    5440,
    55041,
    6080,
    5760,
    54849,
    53761,
    4800,
    4992,
    54081,
    4352,
    53697,
    53377,
    4160,
    61441,
    12480,
    12672,
    61761,
    13056,
    62401,
    62081,
    12864,
    13824,
    63169,
    63361,
    14144,
    62721,
    13760,
    13440,
    62529,
    15360,
    64705,
    64897,
    15680,
    65281,
    16320,
    16000,
    65089,
    64001,
    15040,
    15232,
    64321,
    14592,
    63937,
    63617,
    14400,
    10240,
    59585,
    59777,
    10560,
    60161,
    11200,
    10880,
    59969,
    60929,
    11968,
    12160,
    61249,
    11520,
    60865,
    60545,
    11328,
    58369,
    9408,
    9600,
    58689,
    9984,
    59329,
    59009,
    9792,
    8704,
    58049,
    58241,
    9024,
    57601,
    8640,
    8320,
    57409,
    40961,
    24768,
    24960,
    41281,
    25344,
    41921,
    41601,
    25152,
    26112,
    42689,
    42881,
    26432,
    42241,
    26048,
    25728,
    42049,
    27648,
    44225,
    44417,
    27968,
    44801,
    28608,
    28288,
    44609,
    43521,
    27328,
    27520,
    43841,
    26880,
    43457,
    43137,
    26688,
    30720,
    47297,
    47489,
    31040,
    47873,
    31680,
    31360,
    47681,
    48641,
    32448,
    32640,
    48961,
    32000,
    48577,
    48257,
    31808,
    46081,
    29888,
    30080,
    46401,
    30464,
    47041,
    46721,
    30272,
    29184,
    45761,
    45953,
    29504,
    45313,
    29120,
    28800,
    45121,
    20480,
    37057,
    37249,
    20800,
    37633,
    21440,
    21120,
    37441,
    38401,
    22208,
    22400,
    38721,
    21760,
    38337,
    38017,
    21568,
    39937,
    23744,
    23936,
    40257,
    24320,
    40897,
    40577,
    24128,
    23040,
    39617,
    39809,
    23360,
    39169,
    22976,
    22656,
    38977,
    34817,
    18624,
    18816,
    35137,
    19200,
    35777,
    35457,
    19008,
    19968,
    36545,
    36737,
    20288,
    36097,
    19904,
    19584,
    35905,
    17408,
    33985,
    34177,
    17728,
    34561,
    18368,
    18048,
    34369,
    33281,
    17088,
    17280,
    33601,
    16640,
    33217,
    32897,
    16448,
)

In [30]:
def twobytestring(input) :
    multiplier = 10 ** 0
    integer = int(float(input)* multiplier)
    formatcode = ">"
    formatcode += "H"

    result = struct.pack(formatcode, integer)
    outstring = str(result, encoding = "latin1")
    assert len(outstring) == 2
    return outstring

def twobytestringlsb(input) :
    multiplier = 10 ** 0
    integer = int(float(input)* multiplier)
    formatcode = "<"
    formatcode += "H"

    result = struct.pack(formatcode, integer)
    outstring = str(result, encoding = "latin1")
    assert len(outstring) == 2
    return outstring

def crccheck (inputstring):
    register = 0xFFFF

    for char in inputstring:
        register = (register >> 8) ^ _CRC16TABLE[(register ^ ord(char)) & 0xFF]

    return twobytestringlsb(register)

In [31]:
ser = serial.Serial(
    port=COM,
    baudrate=9600,
    parity=serial.PARITY_NONE,
    stopbits=serial.STOPBITS_ONE,
    bytesize=serial.EIGHTBITS,
    timeout = 1
)

slaveaddress = 1
functioncode = 3
registeraddress = 0x0000
registernumber = 0x0008

first_part = (chr(slaveaddress) + chr(functioncode) + twobytestring(registeraddress) + twobytestring(registernumber))
request = first_part + crccheck(first_part)
request

'\x01\x03\x00\x00\x00\x08D\x0c'

In [32]:
separator = " "

byte_representions = []
for char in request:
    byte_representions.append("{0:02X}".format(ord(char)))
separator.join(byte_representions).strip()

'01 03 00 00 00 08 44 0C'

In [33]:
request = bytes(request, encoding="latin1")
ser.write(request)
out = 0
time.sleep(1)
out = ser.read(21)
out = str(out,encoding = "latin1")
ser.close()
out

'\x01\x03\x10\x034\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x90¿'

In [34]:
separator = " "

byte_representions = []
for char in out:
    byte_representions.append("{0:02X}".format(ord(char)))
separator.join(byte_representions).strip()

'01 03 10 03 34 00 00 00 00 00 00 00 00 00 00 00 00 00 00 90 BF'

In [35]:
first_databyte_number = 3
last_databyte_number = len(out) - 2
payload = out[first_databyte_number:last_databyte_number]

separator = " "

byte_representions = []
for char in payload:
    byte_representions.append("{0:02X}".format(ord(char)))
separator.join(byte_representions).strip()

'03 34 00 00 00 00 00 00 00 00 00 00 00 00 00 00'

In [36]:
def translate(input):
    formatcode = ">"
    formatcode += "H"
    packed = bytes(input,encoding = "latin1")
    val = struct.unpack(formatcode, packed)[0]
    divisor = 10 ** 1
    val = val / float(divisor)
    return val

In [41]:
bytenumberperregister = 2
results = []
for i in range(8):
    byteplace = i * bytenumberperregister
    registerdata = payload[byteplace : byteplace + bytenumberperregister]
    results.append(translate(registerdata)*10)
print(results)

[820.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [42]:
hasilakhir = []
for i in results:
    I = i*20/4095
    hasilakhir.append(I)
print(hasilakhir)

[4.004884004884005, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [43]:
instrument = minimalmodbus.Instrument(COM, 1, debug =True)
instrument.serial.baudrate = 9600
instrument.serial.bytesize = 8
instrument.serial.parity = serial.PARITY_NONE
instrument.serial.stopbits = 1
instrument.serial.timeout = 1
instrument.mode = minimalmodbus.MODE_RTU
instrument.close_port_after_each_call = True
data = instrument.read_registers(registeraddress = 0x0000, number_of_registers = 8, functioncode = 3)
data

MinimalModbus debug mode. Serial port COM7 already exists
MinimalModbus debug mode. Serial port COM7 is closed. Opening.
MinimalModbus debug mode. Will write to instrument (expecting 21 bytes back): 01 03 00 00 00 08 44 0C (8 bytes)
MinimalModbus debug mode. Clearing serial buffers for port COM7
MinimalModbus debug mode. No sleep required before write. Time since previous read: 17110.00 ms, minimum silent period: 4.01 ms.
MinimalModbus debug mode. Closing port COM7
MinimalModbus debug mode. Response from instrument: 01 03 10 03 34 00 00 00 00 00 00 00 00 00 00 00 00 00 00 90 BF (21 bytes), roundtrip time: 0.1 ms. Timeout for reading: 0.0 ms.



[820, 0, 0, 0, 0, 0, 0, 0]